In [ ]:
#서버 ////////////////////
import mysql.connector as my
from flask import Flask, render_template, request
from urllib.request import urlopen
import urllib.request as req
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

#import datetime
from datetime import datetime

mydb=my.connect(
    host="192.168.0.21",
    user="bigdata",
    passwd="123123",
    db="project"
)
print(mydb)

app=Flask(__name__)

@app.route('/') #라즈베리파이IP:5000/로 접속시 index.html으로 연결
def index():
	return render_template('index2.html')

@app.route('/about') 
def about():
	return render_template('about.html')

@app.route('/main') 
def main():
	return render_template('main.html')

@app.route('/search/<txt>')
def search_stock(txt):
	print(txt)
	sql = "select name from stoke where name like '%{}%'".format(txt)
	res = get_list(sql)
	
	return res

@app.route('/analysis/<name>')
def analysis(name):
	print(name)
	
	res = stock_analysis(name)
	
	return "1"



@app.route('/detail/<name>')
def detail_stock(name):
	print(name)
	code = get_code(name)
	res = get_stock_info(code,name)
	
	return res

def get_list(sql):
    mycursor=mydb.cursor()
    mycursor.execute(sql)
    res=mycursor.fetchall()
    
    stock_list = "";
    for x in res:
        name = str(x)[2:-3]
        stock_list = stock_list + "," + name
    return stock_list

def get_code(name):
    mycursor=mydb.cursor()
    sql = "select code from stoke where name='{0}'".format(name)
    mycursor.execute(sql)
    res=mycursor.fetchone()

    code = str(res)[2:-3]
    return code

def get_stock_info(code,name):
    url = "https://finance.naver.com/item/main.nhn?code={}".format(code)
    html=urlopen(url)

    #HTML 분석하기
    soup=BeautifulSoup(html.read(),"html.parser")
    
    info_list=soup.find_all(class_="rate_info")
    data = info_list[0]
    data_list = data.text.split("\n")

    info = ""

    for x in data_list:
        if(x !=''):
            info += x+"@"
    info += code+"@"
    
    
    
    
    try:
        table = get_stock_table(code)
    except:
        table = " "
        
    info += table+"@"
    
    
    cpresult = searchCpPegInfo(name)
    cpname = searchCheck(cpresult)
    
    info += cpname+"@"
    #print(info)
    
    
    return info

def searchCpPegInfo(name):
    mycursor=mydb.cursor()
    mycursor.execute("select peg from stock_data_table where cp_name = '{}'".format(name))
    myresult=mycursor.fetchall()
    return myresult

def searchCheck(cpresult):
    if len(cpresult) == 0:
        cpname = "PEG 값이 없습니다. 신규 상장주이거나 투자대상으로 적합하지 않습니다."
        return cpname
    
    sliceindex = str(cpresult).find(",")
    cpname = str(cpresult)[2:sliceindex]
    return cpname


def get_stock_table(code):
   #cpcode = '005930'
   #cpname = "삼성전자"
    URL = "https://finance.naver.com/item/main.nhn?code={code}".format(code=code)

    html = requests.get(URL).text

    soup = BeautifulSoup(html, 'html.parser')

    finance_html = soup.select_one('div.section.cop_analysis div.sub_section')

    
    #print(finance_html)

    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]

    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]

    finance_data = [item.get_text().strip() for item in finance_html.select('td')]



    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)

    finance_date = annual_date + quarter_date


    finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index,
                          columns=finance_date)

    return finance.to_html()



def stock_analysis(name):
    now = datetime.now()
    today = "{0}-{1}-{2}".format(now.year, now.month, now.day-3)
    #name = "LG전자"
    naming = "2019-6-12/{name}.csv".format(today=today, name=name)
    data = pd.read_csv(naming,  engine='python')
    data.head()
    high_prices = data['고가'].values
    low_prices = data['저가'].values
    mid_prices = (high_prices + low_prices) / 2
    seq_len = 50
    sequence_length = seq_len + 1
    result = []
    for index in range(len(mid_prices) - sequence_length):
        result.append(mid_prices[index: index + sequence_length])
    

    
    normalized_data = []
    for window in result:
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
        
    result = np.array(normalized_data)
    
    # split train and test data
    row = int(round(result.shape[0] * 0.9))
    train = result[:row, :]
    np.random.shuffle(train)
    
    x_train = train[:, :-1]
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    y_train = train[:, -1]
    
    x_test = result[row:, :-1]
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    y_test = result[row:, -1]
    
    x_train.shape, x_test.shape


    model = Sequential() 
    model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='rmsprop')
    model.summary()


    model.fit(x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=10,
        epochs=20)



    pred = model.predict(x_test)

    fig = plt.figure(facecolor='white', figsize=(20, 10))
    ax = fig.add_subplot(111)
    ax.plot(y_test, label='True')
    ax.plot(pred, label='Prediction')
    ax.legend()
    img_name = "static/stock_img/{}.png".format(name)
    plt.savefig(img_name, dpi=350)
    plt.show()
    return "1"




if __name__=='__main__':
	print('WebServer Start')
	app.run(host='localhost')


Using TensorFlow backend.


WebServer Start
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jun/2019 10:13:51] "GET /main HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 10:13:51] "GET /static/css/images/bg01.png HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 10:13:52] "GET /static/css/fonts/fontawesome-webfont.woff?v=3.0.1 HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 10:13:52] "GET /static/css/fonts/fontawesome-social-webfont.woff HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 10:13:52] "GET /static/css/fonts/fontawesome-webfont.ttf?v=3.0.1 HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 10:13:52] "GET /static/css/fonts/fontawesome-social-webfont.ttf HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 10:13:54] "GET /search/%EC%82%BC%EC%84%B1 HTTP/1.1" 200 -


삼성
삼성물산


127.0.0.1 - - [24/Jun/2019 10:13:56] "GET /detail/%EC%82%BC%EC%84%B1%EB%AC%BC%EC%82%B0 HTTP/1.1" 200 -


삼성물산
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 1043 samples, validate on 116 samples
Epoch 1/20
1043/1043 [==============================] - 5s 5ms/step - loss: 0.0022 - val_loss: 0.0132
Epoch 2/20
1043/1043 [==============================] - 4s 4ms/step - loss: 0.0011 - val_loss: 0.0143
Epoch 3/20
1043/1043 [====

<Figure size 2000x1000 with 1 Axes>

127.0.0.1 - - [24/Jun/2019 10:15:22] "GET /analysis/%EC%82%BC%EC%84%B1%EB%AC%BC%EC%82%B0 HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 10:15:22] "GET /static/stock_img/%EC%82%BC%EC%84%B1%EB%AC%BC%EC%82%B0.png HTTP/1.1" 200 -
[2019-06-24 10:36:17,850] ERROR in app: Exception on /detail/삼성SDI [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Progr

삼성SDI


In [ ]:
from flask import Flask

app=Flask(__name__)

@app.route('/')
def index():
	return 'this is main Page'

@app.route('/sub')
def sub():
	return 'this is sub Page'

@app.route('/led/<state>')
def led(state):
	if state == 'on' or state =='off':
		str = 'Led is now %s.' %state
	else:
		str = 'InValid Pages'
	return str

if __name__=='__main__':
	print('WebServer Start')
	app.run(host='0.0.0.0')

In [ ]:
from flask import Flask, render_template, request

app=Flask(__name__)

@app.route('/') #라즈베리파이IP:5000/로 접속시 index.html으로 연결
def index():
	return render_template('index.html')


@app.route('/param',methods=['GET','POST'])
def params():
	if request.method =='GET':
		paramv = request.args.get('status','')
		print("status:"+paramv)
		return paramv

if __name__=='__main__':
	print('WebServer Start')
	app.run(host='0.0.0.0')


In [ ]:
import mysql.connector as my


mydb=my.connect(
    host="192.168.0.21",
    user="bigdata",
    passwd="123123",
    db="project"
)
print(mydb)
mycursor=mydb.cursor()
sql = "select name from stoke where name like '%삼성%'"
mycursor.execute(sql)
res=mycursor.fetchall()

#print(res)
stoke_list = list()
for x in res:
    name = str(x)[2:-3]
    stoke_list.append(name)

print(stoke_list)

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html=urlopen("https://finance.naver.com/item/main.nhn?code=005930")

#HTML 분석하기
soup=BeautifulSoup(html.read(),"html.parser")
    
info_list=soup.find_all(class_="rate_info")
data = info_list[0]
data_list = data.text.split("\n")

print(data)

info = ""

for x in data_list:
    if(x !=''):
        info += x+"\n"
        
print(info)


In [ ]:
rate_info

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html=urlopen("https://finance.naver.com/item/main.nhn?code=005930")

#HTML 분석하기
soup=BeautifulSoup(html.read(),"html.parser")
    
info=soup.find_all(class_="rate_info")
data = info[0]

print(type(data))
#t_list = list(data.dl)
#today = t_list[3].text[6:-4]
#up_down = t_list[5].text
#per = t_list[7].text



t_list2 = list(data.table)

yest = t_list2[5].em.span.text
#high = 

#print()
print(list(t_list2[5])[3])
print("========================================================")
print(data)




#for x in info[0]:
#    data.append(x)

#print(type(data))
#res = tuple(data)
#print(type(res))
#print(res)

In [ ]:

import numpy as np
import pandas as pd
import urllib.request as req
import requests
def get_stock_table(code):
   #cpcode = '005930'
   #cpname = "삼성전자"
    URL = "https://finance.naver.com/item/main.nhn?code={code}".format(code=code)

    html = requests.get(URL).text

    soup = BeautifulSoup(html, 'html.parser')

    finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]

    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]

    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]

    finance_data = [item.get_text().strip() for item in finance_html.select('td')]



    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)

    finance_date = annual_date + quarter_date


    finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index,
                          columns=finance_date)

    return finance.to_html()


res = get_stock_table("005930")
print(type(res))
print(res)